**１.driveマウント**

In [1]:
# @title
# === 1) Google Drive マウント ===
from google.colab import drive
drive.mount("/content/drive")

# === 2) 作業ディレクトリ設定 ===
import os
BASE_DIR = "/content/drive/MyDrive/Inventory_monitoring"
os.chdir(BASE_DIR)

print("現在の作業ディレクトリ:", os.getcwd())
print("ディレクトリ構成:")

# サブフォルダの内容を一覧表示
for folder in ["sql", "data", "util"]:
    path = os.path.join(BASE_DIR, folder)
    if os.path.exists(path):
        files = os.listdir(path)
        print(f"  ┣ {folder}/ ({len(files)} files)")
        for f in files[:10]:  # 上位10件まで表示（多い場合は省略）
            print(f"     ┗ {f}")
        if len(files) > 10:
            print("     ...")
    else:
        print(f"  ┣ {folder}/ (not found)")

# === 3) Google アカウント認証 ===
from google.colab import auth
auth.authenticate_user()
print("Google 認証 OK")

# === 4) BigQuery クライアント初期化 ===
from google.cloud import bigquery
PROJECT_ID = "tential-data-prd"
bq = bigquery.Client(project=PROJECT_ID)
print(f"BigQuery 接続 OK: {bq.project}")


Mounted at /content/drive
現在の作業ディレクトリ: /content/drive/MyDrive/Inventory_monitoring
ディレクトリ構成:
  ┣ sql/ (6 files)
     ┗ 1.sales.sql
     ┗ 3.purchase.sql
     ┗ 4,purchase_plan.sql
     ┗ 5.invent_eom.sql
     ┗ 6.transfer.sql
     ┗ 2.revenue.sql
  ┣ data/ (1 files)
     ┗ 仕入予定(未登録分).gsheet
  ┣ util/ (6 files)
     ┗ common.py
     ┗ init.py
     ┗ __pycache__
     ┗ bq_io.py
     ┗ forecasting.py
     ┗ psi.py
Google 認証 OK
BigQuery 接続 OK: tential-data-prd


**2.データフレーム格納**

In [2]:
# @title
# === 必要モジュール ===
from pathlib import Path
from datetime import date
from util.bq_io import run_sql_file_to_df  # BigQuery 実行関数

# === SQL ファイル定義 ===
SQL_FILES = {
    "sales":         Path("sql/1.sales.sql"),
    "revenue":       Path("sql/2.revenue.sql"),
    "purchase":      Path("sql/3.purchase.sql"),
    "purchase_plan": Path("sql/4,purchase_plan.sql"),
    "invent_eom":    Path("sql/5.invent_eom.sql"),
    "transfer":      Path("sql/6.transfer.sql"),
}

# === 共通クエリパラメータ ===
QUERY_PARAMS = {
    "today": date.today().isoformat(),  # 例: '2025-10-28'
}

# === 実行＆確認 ===
frames = {}
for key, path in SQL_FILES.items():
    df = run_sql_file_to_df(path, QUERY_PARAMS)
    frames[key] = df
    print(f"[OK] {key:14s} <- {path.name:30s} shape={df.shape}")


[OK] sales          <- 1.sales.sql                    shape=(14605, 7)
[OK] revenue        <- 2.revenue.sql                  shape=(3609, 4)
[OK] purchase       <- 3.purchase.sql                 shape=(2607, 4)
[OK] purchase_plan  <- 4,purchase_plan.sql            shape=(2001, 4)
[OK] invent_eom     <- 5.invent_eom.sql               shape=(10386, 4)
[OK] transfer       <- 6.transfer.sql                 shape=(937, 5)
